In [1]:
import requests
import json
import pandas as pd

# Demo: Sending requests

This demo aims at showing how requests can be sent to the server.
Following the instructions shown in the [README.md file](../README.md), the `app.py` (which runs the server) should have been successfxully executed.
We then configure these variables (run the following cell):

In [2]:
domain_name="localhost"
port=3000
auth_token = json.dumps(
            {
                "email":"1000@gmail.com",
                 "password":"qwert2000"
            })

## The root endpoint

This endpoint sends a hello world message.

The URL follows this structure:

`http://{{domain_name}}:{{port}}`

The table below shows examples of values each variable can take:

| Variable | Value |
| --- | --- |
| domain_name | localhost |
| port | 3000 |

In [3]:
url = f"http://{domain_name}:{port}/"
payload = ""
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


    <h1>Hello world! </h1>  <!-- Display a header with the welcome message -->
    Welcome to the Data Access System
    for Malaria Awareness in Rwanda!  <!-- Display a descriptive welcome message -->
    


### Remark:
Given that the above section's code can be strategised, from this part onwards, this function will be used to ease sending requests:

In [4]:
def query(table_name="patient",uri="",payload="",headers={},action="GET"):
    url = f"http://{domain_name}:{port}/tables/{table_name}{uri}"
    print(url)

    response = requests.request(action, url, headers=headers, data=payload)

    return(json.loads(response.text))

## Retrieve a table

URL pattern: `http://{{domain_name}}:{{port}}/tables/{{table_name}}`
This endpoint retrieves the whole table.

The table below shows examples of values each variable can take:

| Variable | Value |
| --- | --- |
| domain_name | localhost |
| port | 3000 |
| table_name | **malaria_results** (can also be any table name present in the database) |

Authentication is needed for access, which is done through inputing the `Authorization` header with credentials as its value (such as `{"email":"`[<code>1000@gmail.com</code>](https://)`","password":"qwert2000"}`) in the request.


In [5]:
malaria_results_dict = query(table_name="malaria_results",
      uri="",
      payload='',
      headers={
          'Authorization': auth_token,
          'Content-Type': 'application/json'
        },
      action="GET")

http://localhost:3000/tables/malaria_results


In [6]:
malaria_results_df = pd.DataFrame.from_dict(malaria_results_dict,orient='index')
malaria_results_df

,blood_test_id,id,malaria_status,parasite_type
0,0,0,positive,pf
1,1,1,negative,pm
2,2,2,negative,pm
3,3,3,positive,pv
4,4,4,positive,pm
...,...,...,...,...
11995,11995,11995,positive,pm
11996,11996,11996,positive,pm
11997,11997,11997,positive,pm
11998,11998,11998,negative,pf


## Sampling

URL pattern: `http://{{domain_name}}:{{port}}/tables/{{table_name}}/sampling`

Returns random samples from the table with a number equal to the "batch_size". It excludes those in "previous_indexes" (see the `Body` of the request example shown here below).

The table below shows examples of values each variable can take:

| Variable | Value |
| --- | --- |
| domain_name | localhost |
| port | 3000 |
| table_name | **patient** (can also be any table name present in the database) |

Authentication is needed for access, which is done through inputing the `Authorization` header with credentials as its value (such as `{"email":"`[<code>1000@gmail.com</code>](https://)`","password":"qwert2000"}`) in the request.

In [7]:
sample_patients_dict = query(table_name="patient",
      uri="/sampling",
      payload=json.dumps({
              "batch_size": 5,
              "previous_indexes": [1,2,3]
            }),
      headers={
          'Authorization': auth_token,
          'Content-Type': 'application/json'
        },
      action="GET")

http://localhost:3000/tables/patient/sampling


In [8]:
sample_patients_dict_df = pd.DataFrame.from_dict(sample_patients_dict["data"],orient='index')
sample_patients_dict_df

,date_of_birth,gender,health_center_id,id,name,village_id
0,"Tue, 10 Oct 1933 11:43:20 GMT",female,600,588,PORYPOYJYK,53140306
1,"Fri, 15 Jun 1990 17:20:00 GMT",male,644,6346,LDGPFLKUIT,42090404
2,"Wed, 19 Jan 1955 07:40:00 GMT",female,768,6575,XRPVSLUKBN,52070108
3,"Wed, 14 Jul 2004 06:46:40 GMT",female,184,7032,OSLCGFXRAL,43090202
4,"Fri, 05 Nov 1993 08:28:20 GMT",male,710,9156,VBBSBLNKAH,55110512


## Time Filtering

URL pattern: `http://{{domain_name}}:{{port}}/tables/{{table_name}}/timefilter`

Returns entries whose values in "date" column are in the specified range (see the `Body` of the request example shown here below).

The table below shows examples of values each variable can take:

| Variable | Value |
| --- | --- |
| domain_name | localhost |
| port | 3000 |
| table_name | **case_cache** (or **blood_test**) |

Authentication is needed for access, which is done through inputing the `Authorization` header with credentials as its value (such as `{"email":"`[<code>1000@gmail.com</code>](https://)`","password":"qwert2000"}`) in the request.

In [9]:
tests_2022_dict = query(table_name="blood_test",
      uri="/timefilter",
      payload=json.dumps({
            "early_date": "2022-01-01",
            "late_date": "2023-01-01"
        }),
      headers={
          'Authorization': auth_token,
          'Content-Type': 'application/json'
        },
      action="GET")

http://localhost:3000/tables/blood_test/timefilter


In [10]:
tests_2022_dict_df = pd.DataFrame.from_dict(tests_2022_dict,orient='index')
tests_2022_dict_df

,date,id,image_url,patient_id
0,"Sat, 01 Jan 2022 00:00:00 GMT",5515,https://9G2eC40rv.com/qCrBgHqn17WHMPQHv,8482
1,"Fri, 07 Jan 2022 00:00:00 GMT",5516,https://7wMIm4.com/t2l4t,712
2,"Tue, 11 Jan 2022 00:00:00 GMT",5517,https://Qa9ua7pBFW1.com/nRtGBf9,4351
3,"Wed, 12 Jan 2022 00:00:00 GMT",5518,https://5EVI37.com/agGOEFra,4993
4,"Thu, 13 Jan 2022 00:00:00 GMT",5519,https://CUhEgkq7ywCVk.com/oMkTcNcnu2T,1170
...,...,...,...,...
90,"Sat, 03 Dec 2022 00:00:00 GMT",5605,https://LVoCINNpSClRaIMGen8.com/Mj05KE,5103
91,"Sun, 04 Dec 2022 00:00:00 GMT",5606,https://9G2eC40rv.com/qCrBgHqn17WHMPQHv,8251
92,"Wed, 07 Dec 2022 00:00:00 GMT",5607,https://7wMIm4.com/t2l4t,6505
93,"Sun, 25 Dec 2022 00:00:00 GMT",5608,https://Qa9ua7pBFW1.com/nRtGBf9,7510


## Column - Key filtering

URL pattern: `http://{{domain_name}}:{{port}}/tables/{{table_name}}/{{column_name}}/{{key}}`

This retrieves particular entries that have the same value in a column of a certain table.

The table below shows examples of values each variable can take:

| Variable | Value |
| --- | --- |
| domain_name | localhost |
| port | 3000 |
| table_name | **patient** (can also be any table name present in the database) |
| column_name | **gender** (can also be any column name present in the database table) |
| key | male (can also be any other value present in the table's column) |

Access to this endpoint needs authentication, which is done through inputing the `Authorization` header with credentials as its value (such as `{"email":"`[<code>1000@gmail.com</code>](https://)`","password":"qwert2000"}`) in the request.

In [11]:
male_patients_dict = query(table_name="patient",
      uri="/gender/male",
      payload="",
      headers={
          'Authorization': auth_token,
          'Content-Type': 'application/json'
        },
      action="GET")

http://localhost:3000/tables/patient/gender/male


In [12]:
male_patients_dict_df = pd.DataFrame.from_dict(male_patients_dict,orient='index')
male_patients_dict_df

,date_of_birth,gender,health_center_id,id,name,village_id
0,"Tue, 01 Aug 2017 19:53:20 GMT",male,264,0,OJMRJGGUNZ,34110108
1,"Mon, 04 Jul 2005 22:53:20 GMT",male,241,1,OANQKDPEIT,22050104
2,"Sun, 31 Dec 1995 13:08:20 GMT",male,586,9,RMDLYGCOFJ,33100306
3,"Thu, 06 Apr 2017 18:11:40 GMT",male,563,11,SMFAVJTZCH,54120409
4,"Thu, 14 May 1959 07:20:00 GMT",male,465,13,VMNOBSNDYN,25010201
...,...,...,...,...,...,...
4958,"Thu, 14 Jul 1938 21:13:20 GMT",male,550,9994,IHSWMBVCCA,55100306
4959,"Sat, 10 Jan 1942 09:43:20 GMT",male,474,9995,JJUPJXXETQ,52050308
4960,"Tue, 15 May 1951 10:06:40 GMT",male,371,9996,FEUPMLHFVP,53130205
4961,"Fri, 15 Mar 2019 00:13:20 GMT",male,291,9997,XJSIYNBLHI,34090201


## Create resource

URL pattern: `http://{{domain_name}}:{{port}}/tables/{{table_name}}/create`

This endpoint is for creating an entry in a database's table.

The URL follows this pattern:

The table below shows examples of values each variable can take:

| Variable | Value |
| --- | --- |
| domain_name | localhost |
| port | 3000 |
| table_name | **patient** (can also be any table name present in the database) |

Access to this endpoint needs authentication, which is done through inputing the `Authorization` header with credentials as its value (such as `{"email":"`[<code>1000@gmail.com</code>](https://)`","password":"qwert2000"}`) in the request.

In [13]:
create_response_dict = query(table_name="patient",
      uri="/create",
      payload=json.dumps({
          "date_of_birth": "Tue, 01 Aug 2017 19:53:20 GMT",
          "gender": "male",
          "health_center_id": 264,
          "name": "OJMRJGGUNZ",
          "village_id": 34110108
        }),
      headers={
          'Authorization': auth_token,
          'Content-Type': 'application/json'
        },
      action="POST")
print(create_response_dict)

http://localhost:3000/tables/patient/create
{'response': 'resource created'}


## Update resource

URL pattern: `http://{{domain_name}}:{{port}}/tables/{{table_name}}/update/{{id}}`

This endpoint is for updating an entry in a database's table.

The URL follows this pattern:

The table below shows examples of values each variable can take:

| Variable | Value |
| --- | --- |
| domain_name | localhost |
| port | 3000 |
| table_name | **patient** (can also be any table name present in the database) |
| id | a primary key to identify a particular entry. |

Access to this endpoint needs authentication, which is done through inputing the `Authorization` header with credentials as its value (such as `{"email":"`[<code>1000@gmail.com</code>](https://)`","password":"qwert2000"}`) in the request.

In [14]:
update_response_dict = query(table_name="patient",
      uri="/update/10000",
      payload=json.dumps({
            "name" : "FROMAA",
            "date_of_birth" : "1940-07-10",
            "gender" :"female",
            "village_id" : 52120102,
            "health_center_id" : 903
        }),
      headers={
          'Authorization': auth_token,
          'Content-Type': 'application/json'
        },
      action="PATCH")
print(update_response_dict)

http://localhost:3000/tables/patient/update/10000
{'response': 'resource updated'}


## Delete resource

URL pattern: `http://{{domain_name}}:{{port}}/tables/{{table_name}}/delete/{{id}}`

This endpoint is for deleting an entry in a database's table.

The table below shows examples of values each variable can take:

| Variable | Value |
| --- | --- |
| domain_name | localhost |
| port | 3000 |
| table_name | **patient** (can also be any table name present in the database) |
| id | a primary key to identify a particular entry. |

Access to this endpoint needs authentication, which is done through inputing the `Authorization` header with credentials as its value (such as `{"email":"`[<code>1000@gmail.com</code>](https://)`","password":"qwert2000"}`) in the request.

In [15]:
delete_response_dict = query(table_name="patient",
      uri="/delete/10000",
      payload='',
      headers={
          'Authorization': auth_token,
          'Content-Type': 'application/json'
        },
      action="DELETE")
print(delete_response_dict)

http://localhost:3000/tables/patient/delete/10000
{'response': 'resource deleted'}
